Download the Dataset
Method 1
Download the Dataset from the following link:
https://www.kaggle.com/jealousleopard/goodreadsbooks/download

In [33]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
import pandas as pd
import surprise

In [34]:
books = pd.read_csv('books.csv')
books.head(2)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,other
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,439785960,9.78044E+12,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,NaN
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,439358078,9.78044E+12,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,NaN


Popularity Based Recommender
Create a function named popularity Recommender and use it to recommend books based on popularity.
Use a weighted rank similar to that used in the IMDB rating example in lesson 2.

In [36]:
books['average_rating'] = pd.to_numeric(books['average_rating'], errors='coerce')
minimum_vote_count = 0.75*books['ratings_count'].max()
mean_rating = books['average_rating'].mean()
books['weighted_rating'] = ((books['ratings_count']/(books['ratings_count'] + minimum_vote_count)) * books['average_rating'] 
+ (minimum_vote_count/(books['ratings_count'] + minimum_vote_count)) * mean_rating)
recommendations = books.sort_values(by = 'weighted_rating', ascending = False).head(5)
recommendations.head(2)


,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,other,weighted_rating
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9.78044E+12,eng,435,2339585,36325,5/1/2004,Scholastic Inc.,NaN,4.187089
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,439785960,9.78044E+12,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,NaN,4.174464


Content based Recommender
Create a function named content Based Recommender and use it to recommend books based on content.
TF-IDF Vectorizer
Use TF-IDF Vectorizer on the author data for each book.
Distance matrix
Choose cosine similarity for pairwise distances comparison

In [38]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [74]:
def AuthorBasedRecommender(title):
    books = pd.read_csv('books.csv')
    
    #instantiate a new Vectorizer object
    tfidf = TfidfVectorizer(stop_words = 'english')
    #replace NaN with empty string and create the index for the authors
    books['title'] = books['title'].fillna('')
    indices = pd.Series(books.index, index = books['title']).drop_duplicates()
    
    #transforms the authors to the tfidf matrix
    tfidf_matrix = tfidf.fit_transform(books['title'])
    distance_matrix = cosine_similarity(tfidf_matrix)
    id_ = indices[title]
    distances = list(enumerate(distance_matrix[id_]))
    distances = sorted(distances, key = lambda x: x[1], reverse = True)
    recommendations = [distance[0] for distance in distances]
    return books['title'].iloc[recommendations][1:6]
AuthorBasedRecommender('Las aventuras de Tom Sawyer') 

2372                     Las aventuras de Sherlock Holmes
6514    The Adventures of Tom Sawyer (Adventures of To...
6515    The Adventures of Tom Sawyer (Adventures of To...
896            Huck Finn and Tom Sawyer Among the Indians
6513    The Adventures of Tom Sawyer and Adventures of...
Name: title, dtype: object